In [1]:
import pandas as pd
import tensorflow as tf
from utils import load_model

In [2]:
with open('./data/dostoewskij/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
sentences = text.split('\n')
batch_size = 325

In [68]:
from collections import defaultdict

In [77]:
batches = []
for i in range(10): # len(sentences)//batch_size):
    batch = sentences[i*batch_size:(i+1)*batch_size]
    batches.append(batch)

In [ ]:
sess = tf.InteractiveSession()
model, transformer = load_model('./save/shm_c1', sess, False, decoding=False, seq_length=64, batch_size=batch_size)

In [4]:
states = []
for i, batch in enumerate(batches):
    states.append(model.calculate_states(sess, transformer, phrases=batch))
    print(i+1, '/', len(batches),  end='\r')

INFO:tensorflow:Restoring parameters from ./save/shm_c1/model.ckpt-58519


In [74]:
import re
pat = re.compile(r'([a-я:, "]+[?!.]+)')
w_pat = re.compile(r'[а-я]+')

with open('words_dictionary.txt', 'r') as f:
    dictionary = f.read().split('_')

def check_sent(seq):
    return len(seq) < 200

def filter_sequence(seq, dictionary):
    candidates = pat.findall(seq)
    selected = []
    for cand in candidates:
        cand = cand.strip()
        words = w_pat.findall(cand)
        keep = True
        for word in words:
            if word not in dictionary:
                keep = False
        if keep:
            selected.append(cand)
    return selected

In [75]:
g = tf.Graph()
with g.as_default():
    sess_sample = tf.Session()
    model_sample, _ = load_model('./save/shm_c1', sess_sample, False, decoding=True, seq_length=201, batch_size=batch_size)

INFO:tensorflow:Restoring parameters from ./save/shm_c1/model.ckpt-58519


In [78]:
sampled_by_seed = defaultdict(list)
for state, batch in zip(states, batches):
    samples = model_sample.loop_sample(sess_sample, transformer, state)
    for sample, seed in zip(samples, batch):
        if check_sent(seed):
            sampled_by_seed[seed] += filter_sequence(sample)

In [79]:
sampled_by_seed

defaultdict(list,
            {'федор михайлович достоевский.': [],
             'белые ночи. иль был он создан для того, чтобы побыть хотя мгновение.': ['и все для меня!'],
             'в соседстве сердца твоего?...': ['отец наш?'],
             'ив.': ['чтото?'],
             'тургеньев.': ['да?'],
             'ночь первая.': ['у друга.', 'господи!'],
             'была чудная ночь, такая ночь, которая разве только и может быть тогда, когда мы молоды, любезный читатель.': [],
             'небо было такое звездное, такое светлое небо, что, взглянув на него, невольно нужно было спросить себя: неужели же могут жить под таким небом разные сердитые и капризные люди?': [],
             'это тоже молодой вопрос, любезный читатель, очень молодой, но пошли его вам господь чаще на душу!..': ['ах, какие мне проклятые!',
              'вскричала она мне, уверяя меня за руку.'],
             'говоря о капризных и разных сердитых господах, я не мог не припомнить и своего благонравного поведения

In [66]:
selected = []
for s in sampled:
    selected += filter_sequence(s)